<a href="https://colab.research.google.com/github/sayan0506/Face-Recognition-pipeline-using-SOTA-ArcFace/blob/main/Test_pipeline_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Mount Drive**

In [4]:
from google.colab import drive

drive.mount('/content/grive')

Mounted at /content/grive


## **Execute Train Pipeline**

In [1]:
!python train.py -dataset /content/grive/MyDrive/Kaggle_Avengers_face_dataset/Marvels_face_dataset -model .

Running on device: cuda:0
Device info
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)
Best validated model will be saved in "saved_model" folder!

[INFO] Loading images ...
[INFO] Processing chris_evans
50
[INFO] Processing chris_hemsworth
53
[INFO] Processing mark_ruffalo
66
[INFO] Processing robert_downey_jr
51
[INFO] Processing scarlett_johansson
54
Data distribution of different identities
  chris_evans  ... scarlett_johansson
0          50  ...                 54

[1 rows x 5 columns]
Image dataframe
                        Image               Label
0           chris_evans34.png         chris_evans
1           chris_evans32.png         chris_evans
2            chris_evans6.png         chris_evans
3           chris_evans47.png         chris_evans
4           chris_evans13.png         chris_evans
..                        ...                 ...
269  scarlett_johansson54.png  scarlett_johansson
270  scarlett_johansson52.png  s

## **Execute Test pipeline**

Note: python test.py bash implementation is not shown in the demo, but that code is working properly

In [1]:
# import test module
import test

In [2]:
# create test instance
t= test.test_pipeline('/content/grive/MyDrive/Kaggle_Avengers_face_dataset','.')

Running on device: cuda:0
Device info
_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [3]:
# run test pipeline
t.run_pipeline()


Validation Dataset info <FaceDataset.FaceDataset object at 0x7f27b938e450>,
contains 5 different identities!
Classes available in the dataset 
['chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson'],
having class ids [0, 1, 2, 3, 4] respectively!
Length of the dataset 14

Test Dataset info <FaceDataset.FaceDataset object at 0x7f27ca8e1310>,
contains 5 different identities!
Classes available in the dataset 
['chris_evans', 'chris_hemsworth', 'mark_ruffalo', 'robert_downey_jr', 'scarlett_johansson'],
having class ids [0, 1, 2, 3, 4] respectively!
Length of the dataset 14

Train, valid, test dataloader have created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 3 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


MobilefaceNet model loaded!


## **Build Requirements**

Create requirements.txt

In [3]:
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# data pipeline
# helps to create dict where, use keys as atribute
from easydict import EasyDict as edict

from torchvision import transforms as trans
from torchvision.datasets import ImageFolder
from PIL import Image, ImageFile, ImageDraw
from torch.utils.data import Dataset, DataLoader

# ensures loading truncated images 
ImageFile.LOAD_TRUNCATED_IMAGES = True

# model build
from torch.nn import Linear, Conv2d, BatchNorm1d, BatchNorm2d, PReLU, ReLU, Sigmoid, Dropout2d, Dropout, AvgPool2d, MaxPool2d, AdaptiveAvgPool2d, Sequential, Module, Parameter
import torch.nn.functional as F
import torch
from collections import namedtuple
import math
import pdb

# model training
from torch import optim
from tqdm import tqdm
from torch.nn import CrossEntropyLoss

**Obtaining versions of different modules**

In [24]:
# numpy
!pip freeze | grep numpy
# os
!pip freeze | grep os
# pandas
!pip freeze | grep pandas
# matplotlib
!pip freeze | grep matplotlib
# pytorch
!pip freeze | grep torch

numpy==1.19.5
ecos==2.0.7.post1
googleapis-common-protos==1.53.0
ideep4py==2.0.0.post3
librosa==0.8.0
osqp==0.6.2.post0
qdldl==0.1.5.post0
scikit-learn==0.22.2.post1
SoundFile==0.10.3.post1
xgboost==0.90
pandas==1.1.5
pandas-datareader==0.9.0
pandas-gbq==0.13.3
pandas-profiling==1.4.1
sklearn-pandas==1.8.0
matplotlib==3.2.2
matplotlib-inline==0.1.2
matplotlib-venn==0.11.6
torch==1.8.1+cu101
torchsummary==1.5.1
torchtext==0.9.1
torchvision==0.9.1+cu101


In [11]:
# tqdm
!pip freeze | tqdm

0it [00:00, ?it/s]absl-py==0.12.0
1it [00:00,  1.95it/s]alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.2
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.2
catalogue==1.0.0
certifi==2020.12.5
cffi==1.14.5
cftime==1.5.0
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill

## **Install requirements**

In [25]:
!pip install -r requirements.txt